
# **Setting Up Ubuntu 24.04 with KDE Plasma Desktop on WSL2 with X410 and GPU Support**

## Introduction

This tutorial will guide you through installing **Ubuntu 24.04** on **WSL2** (Windows Subsystem for Linux), configuring it with the **KDE Plasma desktop environment**, and using **X410** as an XServer on a **Windows 11** host. We will also enable GPU acceleration and provide steps to configure the system for smooth graphical performance.

WSL2 brings a near-native Linux experience to Windows, and with the ability to run full graphical Linux desktops like KDE Plasma, the experience is even closer to a traditional Linux environment. With GPU support in WSL2, it’s also possible to run GPU-accelerated applications, making it a powerful development environment for software developers.

This tutorial is targeted at developers who want to create a full Linux desktop environment inside their Windows environment without dual-booting or using a separate virtual machine.
        


## Step 1: Install WSL2 and Ubuntu 24.04

### 1.1. Enable WSL2 and Virtualization
Before installing Ubuntu 24.04, ensure that WSL2 is enabled on your Windows machine.

1. Open **PowerShell** as an Administrator and run:
   ```bash
   wsl --install
   ```

2. Set WSL to use version 2 by default:
   ```bash
   wsl --set-default-version 2
   ```

3. If Ubuntu 24.04 is not listed, you can manually install it by running:
   ```bash
   wsl --install -d Ubuntu-24.04
   ```

4. After installation, set your default distribution:
   ```bash
   wsl --set-default Ubuntu-24.04
   ```
        


## Step 2: Configure WSL Networking for XServer

In WSL2, each Linux instance runs in its own virtualized network environment, and we need to configure it to communicate with the **XServer** on Windows.

### 2.1. Configure Mirror Networking

Mirror networking allows the Linux instance to communicate directly with the Windows host, and it requires exporting the correct display IP to render graphical apps.

- Use the loopback address `127.0.0.1` to tell the system to render graphical apps on the XServer running on Windows:
  
  ```bash
  echo 'export DISPLAY=127.0.0.1:0' >> ~/.profile
  ```

- Ensure that the `.profile` is sourced on login by adding this to your `~/.bashrc`:
  
  ```bash
  echo 'source ~/.profile' >> ~/.bashrc
  ```

- Now, each time you log in to WSL2, it will set the display correctly for XServer.
        
- On the Windows host, edit %username%/.wslconfig . If the file does not exist, create a new file.  Add the following:
  ```bash
  [wsl2]
  guiApplications=false
 
  networkingMode=mirrored
  ```
  This enables mirror mode, which mirrors all local network connection and the guest to communicate with the host via the loopback adapter. 
  
- Save the file and then restart WSL with:

  ```bash
    wsl --shutdown
    wsl
  ```


## Step 3: Install KDE Plasma Desktop on Ubuntu 24.04

### 3.1. Install the KDE Plasma Desktop

To install the KDE Plasma desktop environment, run the following commands in your Ubuntu WSL terminal:

```bash
sudo apt update
sudo apt install kde-plasma-desktop -y
```

If you want a full KDE desktop environment with applications, use this command instead:

```bash
sudo apt install kde-full -y
```

### 3.2. Start KDE Plasma

Once installed, KDE can be started manually by running the following command inside your WSL2 terminal:

```bash
startplasma-x11
```
        


## Step 4: Set Up X410 on Windows 11

X410 is an XServer for Windows that allows running Linux GUI apps. Here's how to configure it for WSL2:

### 4.1. Install X410
1. Download and install **X410** from the Microsoft Store or [official website](https://x410.dev/).
2. Once installed, launch X410 in **Floating Desktop** mode:
   - Right-click on the X410 icon in the system tray and select **Floating Desktop**.

### 4.2. Configure X410 to Listen on Loopback Interface

By default, X410 will listen on the loopback interface (127.0.0.1). To confirm, check that X410 is running in **Floating Desktop** mode and that it’s not restricted by any firewall rules.
        


## Step 5: Enable GPU Acceleration

WSL2 now supports GPU acceleration, which can significantly improve performance for graphical applications.

### 5.1. Install NVIDIA Drivers

If you have an NVIDIA GPU, you’ll need the latest Windows Display Driver Model (WDDM) drivers installed. Ensure you download the correct drivers from the [NVIDIA website](https://www.nvidia.com/Download/index.aspx).

Inside Ubuntu, install the NVIDIA CUDA drivers and OpenGL libraries:

```bash
sudo apt install nvidia-cuda-toolkit
sudo apt install libnvidia-gl-<version>
```

### 5.2. Configure GPU for OpenGL

Edit your `~/.profile` to include the following line, which will force KDE and other OpenGL applications to use the NVIDIA GPU:

```bash
echo 'export __GLX_VENDOR_LIBRARY_NAME=nvidia' >> ~/.profile
```

Source your `.profile` again:

```bash
source ~/.profile
```
        


## Step 6: Create a Script to Launch KDE Plasma Desktop

Create a script to automate the startup of KDE:

```bash
nano ~/start-kde.sh
```

Add the following content:

```bash
#!/bin/bash
export DISPLAY=127.0.0.1:0
startplasma-x11
```

Make the script executable:

```bash
chmod +x ~/start-kde.sh
```
        


## Step 7: Create a Windows Batch File to Launch X410 and WSL2

Create a Windows batch file that will launch X410 in **Floating Desktop** mode, start WSL2, and run the KDE Plasma environment:

1. Open **Notepad** and paste the following:

```batch
@echo off
REM Start X410 in Floating Desktop Mode
"C:\Program Files\X410A0.exe" /wm

REM Give X410 some time to start up
timeout /t 5 /nobreak >nul

REM Launch WSL and KDE Plasma
wsl -d Ubuntu-24.04 -e bash -c "~/start-kde.sh"
exit
```

2. Save the file as `start_kde.bat`.

Now, simply double-click this batch file to start X410 and KDE Plasma from Windows.
        


## Step 8: Troubleshooting Display Issues

### 8.1. Alternative Method for Exporting Display IP
If using `127.0.0.1:0` for the display doesn’t work, try using the host IP address. This can be dynamically extracted from WSL2’s `/etc/resolv.conf` file:

```bash
export DISPLAY=$(grep nameserver /etc/resolv.conf | awk '{print $2}'):0
```

Add this to your `~/.profile` if needed:

```bash
echo 'export DISPLAY=$(grep nameserver /etc/resolv.conf | awk '{print $2}'):0' >> ~/.profile
```

### 8.2. XServer Not Responding
If X410 isn’t responding, ensure your firewall allows inbound connections to the XServer and that WSL2 has access to the Windows host network.

- Check firewall settings and allow traffic on port **6000** for X11.
  
---
        


## Conclusion

By following these steps, you've successfully installed and configured **Ubuntu 24.04 with KDE Plasma Desktop** on **WSL2**, and set up **X410** for seamless graphical interaction. This setup, complete with **GPU acceleration**, provides a powerful Linux desktop environment directly within your Windows development environment. Whether you're developing GUI-based applications, working on GPU-accelerated tasks, or just prefer the KDE Plasma interface, this setup allows you to work efficiently without leaving the comfort of your Windows host.
        